<a href="https://colab.research.google.com/github/HussainAhmed202/my-bioinformatics-journal/blob/main/Projects/Protein_Function_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Neural Network-Based Multi-Label Classifier for Protein Function Prediction

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!pip install unrar
!unrar x  "/content/drive/MyDrive/Dataset.rar" "/content"



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/Dataset.rar

Creating    /content/Test Dataset                                     OK
Extracting  /content/Test Dataset/Act_test.csv                             0%  1%  2%  OK 
Extracting  /content/Test Dataset/Bac_test.csv                             2%  OK 
Extracting  /content/Test Dataset/Chl_test.csv                             3%  OK 
Extracting  /content/Test Dataset/Cya_test.csv                             4%  OK 
Extracting  /content/Test Dataset/Fir_test1.csv                            5%  6%  7%  8%  OK 
Extracting  /content/Test Dataset/Fir_test2.csv                            9% 10% 11%  OK 
Extracting  /content/Test Dataset/Fuso_test.csv                           11%  OK 
Extracting  /content/Test Dataset/Pro_test1.csv                           12% 13% 14%

In [5]:
!nvidia-smi

Fri Feb 21 04:18:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Import all Libraries

In [3]:
# import cudf
# %load_ext cudf.pandas
import pandas as pd
#import sys


# Import all the datasets

## Actinobacteria

In [4]:
data_Actinobacteria1 = pd.read_csv('/content/Train Dataset/Act_train1.csv')
data_Actinobacteria1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9323), int64(2306), object(3)
memory usage: 440.0 MB


The memory usage for this dataset is **440MB**. We can reduce this by using other int and float datatypes for smaller number

In Pandas for **Integer** data the default dtype is **int64**

* **int64** is a signed 64-bit integer range -(2^63) -> (2^63 - 1)

Other options are **int8** and **int16**
* **int16** is a signed 16-bit integer range -(2^15) -> (2^15 - 1)
* **int8** is a signed 8-bit integer range -(2^7) -> (2^7 - 1)


In [5]:
# define the ranges
min_val_8bit, max_val_8bit = -2**7, 2**7 - 1
min_val_16bit, max_val_16bit = -2**15, 2**15 - 1

In [6]:
# Goal: Iterate over all integer columns
# If their max and min value can be represented in 8-bit binary then replace with int8
# If their max and min value can be represented in 16-bit binary then replace with int16

int_cols = data_Actinobacteria1.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Actinobacteria1[col].max() < max_val_8bit) and (data_Actinobacteria1[col].min() > min_val_8bit):
    data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('int8')
  elif (data_Actinobacteria1[col].max() < max_val_16bit) and (data_Actinobacteria1[col].min() > min_val_16bit):
      data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('int16')

In [7]:
# check change in size
data_Actinobacteria1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9323), int16(1), int64(2), int8(2303), object(3)
memory usage: 364.2 MB


In Pandas for **Floating Point** data the default dtype is float64. Floating point numbers are stored using **IEEE 754** standard

* **float64** is double precision floating point number ranging -1.7e+308 to +1.7e+308.

Other options are **float16** and **float32**
* **float32** is a single precision range from -3.4e+38 to 3.4e+38.

* **float16** is a half  precision range from -(6.55×104) to +(6.55×104)




In [8]:
# define the ranges
min_val_16bit, max_val_16bit = -6.55*(10**4), 6.55*(10**4)
min_val_32bit, max_val_32bit = -3.4*(10**38), 3.4*(10**38)

In [9]:
# Goal: Iterate over all float columns
# If their max and min value can be represented in 16-bit binary then replace with float16
# If their max and min value can be represented in 32-bit binary then replace with float32

float_cols = data_Actinobacteria1.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Actinobacteria1[col].max() < max_val_16bit) and (data_Actinobacteria1[col].min() > min_val_16bit):
      data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('float16')
  elif (data_Actinobacteria1[col].max() < max_val_32bit) and (data_Actinobacteria1[col].min() > min_val_32bit):
    data_Actinobacteria1[col] = data_Actinobacteria1[col].astype('float32')


In [10]:
# check change in size
data_Actinobacteria1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9322), float32(1), int16(1), int64(2), int8(2303), object(3)
memory usage: 101.1 MB


The size has been reduced to **101.1MB**

In [11]:
data_Actinobacteria2 = pd.read_csv('/content/Train Dataset/Act_train2.csv')
data_Actinobacteria2.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9323), int64(2306), object(3)
memory usage: 440.1 MB


In [12]:
# reduce any int64 columns
int_cols = data_Actinobacteria2.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Actinobacteria2[col].max() < max_val_8bit) and (data_Actinobacteria2[col].min() > min_val_8bit):
    data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('int8')
  elif (data_Actinobacteria2[col].max() < max_val_16bit) and (data_Actinobacteria2[col].min() > min_val_16bit):
      data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('int16')


# reduce any float64 columns
float_cols = data_Actinobacteria2.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Actinobacteria2[col].max() < max_val_16bit) and (data_Actinobacteria2[col].min() > min_val_16bit):
      data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('float16')
  elif (data_Actinobacteria2[col].max() < max_val_32bit) and (data_Actinobacteria2[col].min() > min_val_32bit):
    data_Actinobacteria2[col] = data_Actinobacteria2[col].astype('float32')



data_Actinobacteria2.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4932 entries, 0 to 4931
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9322), float32(1), int16(1), int64(2), int8(2303), object(3)
memory usage: 101.1 MB


## Bacteroidetes

In [13]:
data_Bacteroidetes = pd.read_csv('/content/Train Dataset/Bac_train.csv')
data_Bacteroidetes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3149 entries, 0 to 3148
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(11629), object(3)
memory usage: 280.9 MB


In [14]:
# reduce any int64 columns
int_cols = data_Bacteroidetes.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Bacteroidetes[col].max() < max_val_8bit) and (data_Bacteroidetes[col].min() > min_val_8bit):
    data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('int8')
  elif (data_Bacteroidetes[col].max() < max_val_16bit) and (data_Bacteroidetes[col].min() > min_val_16bit):
      data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('int16')


# reduce any float64 columns
float_cols = data_Bacteroidetes.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Bacteroidetes[col].max() < max_val_16bit) and (data_Bacteroidetes[col].min() > min_val_16bit):
      data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('float16')
  elif (data_Bacteroidetes[col].max() < max_val_32bit) and (data_Bacteroidetes[col].min() > min_val_32bit):
    data_Bacteroidetes[col] = data_Bacteroidetes[col].astype('float32')



data_Bacteroidetes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3149 entries, 0 to 3148
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(11626), float32(3), object(3)
memory usage: 71.4 MB


## Chlamydiae

In [15]:
data_Chlamydiae = pd.read_csv('/content/Train Dataset/Chl_train.csv')
data_Chlamydiae.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9214), int64(2415), object(3)
memory usage: 116.8 MB


In [16]:
# reduce any int64 columns
int_cols = data_Chlamydiae.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Chlamydiae[col].max() < max_val_8bit) and (data_Chlamydiae[col].min() > min_val_8bit):
    data_Chlamydiae[col] = data_Chlamydiae[col].astype('int8')
  elif (data_Chlamydiae[col].max() < max_val_16bit) and (data_Chlamydiae[col].min() > min_val_16bit):
      data_Chlamydiae[col] = data_Chlamydiae[col].astype('int16')


# reduce any float64 columns
float_cols = data_Chlamydiae.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Chlamydiae[col].max() < max_val_16bit) and (data_Chlamydiae[col].min() > min_val_16bit):
      data_Chlamydiae[col] = data_Chlamydiae[col].astype('float16')
  elif (data_Chlamydiae[col].max() < max_val_32bit) and (data_Chlamydiae[col].min() > min_val_32bit):
    data_Chlamydiae[col] = data_Chlamydiae[col].astype('float32')



data_Chlamydiae.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9213), float32(1), int16(1), int64(2), int8(2412), object(3)
memory usage: 26.7 MB


## Cyanobacteria

In [17]:
data_Cyanobacteria = pd.read_csv('/content/Train Dataset/Cya_train.csv')
data_Cyanobacteria.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5459 entries, 0 to 5458
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9318), int64(2311), object(3)
memory usage: 487.0 MB


In [18]:
# reduce any int64 columns
int_cols = data_Cyanobacteria.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Cyanobacteria[col].max() < max_val_8bit) and (data_Cyanobacteria[col].min() > min_val_8bit):
    data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('int8')
  elif (data_Cyanobacteria[col].max() < max_val_16bit) and (data_Cyanobacteria[col].min() > min_val_16bit):
      data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('int16')


# reduce any float64 columns
float_cols = data_Cyanobacteria.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Cyanobacteria[col].max() < max_val_16bit) and (data_Cyanobacteria[col].min() > min_val_16bit):
      data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('float16')
  elif (data_Cyanobacteria[col].max() < max_val_32bit) and (data_Cyanobacteria[col].min() > min_val_32bit):
    data_Cyanobacteria[col] = data_Cyanobacteria[col].astype('float32')



data_Cyanobacteria.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5459 entries, 0 to 5458
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9317), float32(1), int16(1), int64(2), int8(2308), object(3)
memory usage: 111.8 MB


## Firmicutes

In [19]:
data_Firmicutes1 = pd.read_csv('/content/Train Dataset/Fir_train1.csv')
data_Firmicutes1.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9329), int64(2300), object(3)
memory usage: 240.7 MB


In [20]:
# reduce any int64 columns
int_cols = data_Firmicutes1.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Firmicutes1[col].max() < max_val_8bit) and (data_Firmicutes1[col].min() > min_val_8bit):
    data_Firmicutes1[col] = data_Firmicutes1[col].astype('int8')
  elif (data_Firmicutes1[col].max() < max_val_16bit) and (data_Firmicutes1[col].min() > min_val_16bit):
      data_Firmicutes1[col] = data_Firmicutes1[col].astype('int16')


# reduce any float64 columns
float_cols = data_Firmicutes1.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Firmicutes1[col].max() < max_val_16bit) and (data_Firmicutes1[col].min() > min_val_16bit):
      data_Firmicutes1[col] = data_Firmicutes1[col].astype('float16')
  elif (data_Firmicutes1[col].max() < max_val_32bit) and (data_Firmicutes1[col].min() > min_val_32bit):
    data_Firmicutes1[col] = data_Firmicutes1[col].astype('float32')



data_Firmicutes1.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9328), float32(1), int16(1), int64(2), int8(2297), object(3)
memory usage: 55.3 MB


In [21]:
data_Firmicutes2 = pd.read_csv('/content/Train Dataset/Fir_train2.csv')
data_Firmicutes2.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5993 entries, 0 to 5992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9329), int64(2300), object(3)
memory usage: 534.3 MB


In [22]:
# reduce any int64 columns
int_cols = data_Firmicutes2.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Firmicutes2[col].max() < max_val_8bit) and (data_Firmicutes2[col].min() > min_val_8bit):
    data_Firmicutes2[col] = data_Firmicutes2[col].astype('int8')
  elif (data_Firmicutes2[col].max() < max_val_16bit) and (data_Firmicutes2[col].min() > min_val_16bit):
      data_Firmicutes2[col] = data_Firmicutes2[col].astype('int16')


# reduce any float64 columns
float_cols = data_Firmicutes2.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Firmicutes2[col].max() < max_val_16bit) and (data_Firmicutes2[col].min() > min_val_16bit):
      data_Firmicutes2[col] = data_Firmicutes2[col].astype('float16')
  elif (data_Firmicutes2[col].max() < max_val_32bit) and (data_Firmicutes2[col].min() > min_val_32bit):
    data_Firmicutes2[col] = data_Firmicutes2[col].astype('float32')



data_Firmicutes2.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5993 entries, 0 to 5992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9328), float32(1), int16(1), int64(2), int8(2297), object(3)
memory usage: 122.5 MB


## Fusobacteria

In [23]:
data_Fusobacteria = pd.read_csv('/content/Train Dataset/Fuso_train.csv')
data_Fusobacteria.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(8414), int64(3215), object(3)
memory usage: 16.7 MB


In [24]:
# reduce any int64 columns
int_cols = data_Fusobacteria.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Fusobacteria[col].max() < max_val_8bit) and (data_Fusobacteria[col].min() > min_val_8bit):
    data_Fusobacteria[col] = data_Fusobacteria[col].astype('int8')
  elif (data_Fusobacteria[col].max() < max_val_16bit) and (data_Fusobacteria[col].min() > min_val_16bit):
      data_Fusobacteria[col] = data_Fusobacteria[col].astype('int16')


# reduce any float64 columns
float_cols = data_Fusobacteria.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Fusobacteria[col].max() < max_val_16bit) and (data_Fusobacteria[col].min() > min_val_16bit):
      data_Fusobacteria[col] = data_Fusobacteria[col].astype('float16')
  elif (data_Fusobacteria[col].max() < max_val_32bit) and (data_Fusobacteria[col].min() > min_val_32bit):
    data_Fusobacteria[col] = data_Fusobacteria[col].astype('float32')



data_Fusobacteria.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(8413), float32(1), int16(1), int64(2), int8(3212), object(3)
memory usage: 3.7 MB


## Proteobacteria

In [25]:
data_Proteobacteria = pd.read_csv('/content/Train Dataset/Pro_train1.csv')
data_Proteobacteria.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9303), int64(2326), object(3)
memory usage: 445.9 MB


In [26]:
# reduce any int64 columns
int_cols = data_Proteobacteria.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Proteobacteria[col].max() < max_val_8bit) and (data_Proteobacteria[col].min() > min_val_8bit):
    data_Proteobacteria[col] = data_Proteobacteria[col].astype('int8')
  elif (data_Proteobacteria[col].max() < max_val_16bit) and (data_Proteobacteria[col].min() > min_val_16bit):
      data_Proteobacteria[col] = data_Proteobacteria[col].astype('int16')


# reduce any float64 columns
float_cols = data_Proteobacteria.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Proteobacteria[col].max() < max_val_16bit) and (data_Proteobacteria[col].min() > min_val_16bit):
      data_Proteobacteria[col] = data_Proteobacteria[col].astype('float16')
  elif (data_Proteobacteria[col].max() < max_val_32bit) and (data_Proteobacteria[col].min() > min_val_32bit):
    data_Proteobacteria[col] = data_Proteobacteria[col].astype('float32')



data_Proteobacteria.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9302), float32(1), int16(1), int64(2), int8(2323), object(3)
memory usage: 102.7 MB


In [27]:
data_Proteobacteria4 = pd.read_csv('/content/Train Dataset/Pro_train4.csv')
data_Proteobacteria4.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4994 entries, 0 to 4993
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9177), int64(2452), object(3)
memory usage: 445.3 MB


In [28]:
# reduce any int64 columns
int_cols = data_Proteobacteria4.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Proteobacteria4[col].max() < max_val_8bit) and (data_Proteobacteria4[col].min() > min_val_8bit):
    data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('int8')
  elif (data_Proteobacteria4[col].max() < max_val_16bit) and (data_Proteobacteria4[col].min() > min_val_16bit):
      data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('int16')


# reduce any float64 columns
float_cols = data_Proteobacteria4.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Proteobacteria4[col].max() < max_val_16bit) and (data_Proteobacteria4[col].min() > min_val_16bit):
      data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('float16')
  elif (data_Proteobacteria4[col].max() < max_val_32bit) and (data_Proteobacteria4[col].min() > min_val_32bit):
    data_Proteobacteria4[col] = data_Proteobacteria4[col].astype('float32')



data_Proteobacteria4.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4994 entries, 0 to 4993
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9176), float32(1), int16(1), int64(2), int8(2449), object(3)
memory usage: 101.4 MB


In [29]:
data_Proteobacteria5 = pd.read_csv('/content/Train Dataset/Pro_train5.csv')
data_Proteobacteria5.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(8455), int64(3174), object(3)
memory usage: 445.0 MB


In [30]:
# reduce any int64 columns
int_cols = data_Proteobacteria5.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Proteobacteria5[col].max() < max_val_8bit) and (data_Proteobacteria5[col].min() > min_val_8bit):
    data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('int8')
  elif (data_Proteobacteria5[col].max() < max_val_16bit) and (data_Proteobacteria5[col].min() > min_val_16bit):
      data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('int16')


# reduce any float64 columns
float_cols = data_Proteobacteria5.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Proteobacteria5[col].max() < max_val_16bit) and (data_Proteobacteria5[col].min() > min_val_16bit):
      data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('float16')
  elif (data_Proteobacteria5[col].max() < max_val_32bit) and (data_Proteobacteria5[col].min() > min_val_32bit):
    data_Proteobacteria5[col] = data_Proteobacteria5[col].astype('float32')



data_Proteobacteria5.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(8455), int16(3), int8(3171), object(3)
memory usage: 97.3 MB


## Spirochaetes

In [31]:
data_Spirochaetes = pd.read_csv('/content/Train Dataset/Spi_train.csv')
data_Spirochaetes.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1940 entries, 0 to 1939
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9144), int64(2485), object(3)
memory usage: 173.1 MB


In [32]:
# reduce any int64 columns
int_cols = data_Spirochaetes.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Spirochaetes[col].max() < max_val_8bit) and (data_Spirochaetes[col].min() > min_val_8bit):
    data_Spirochaetes[col] = data_Spirochaetes[col].astype('int8')
  elif (data_Spirochaetes[col].max() < max_val_16bit) and (data_Spirochaetes[col].min() > min_val_16bit):
      data_Spirochaetes[col] = data_Spirochaetes[col].astype('int16')


# reduce any float64 columns
float_cols = data_Spirochaetes.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Spirochaetes[col].max() < max_val_16bit) and (data_Spirochaetes[col].min() > min_val_16bit):
      data_Spirochaetes[col] = data_Spirochaetes[col].astype('float16')
  elif (data_Spirochaetes[col].max() < max_val_32bit) and (data_Spirochaetes[col].min() > min_val_32bit):
    data_Spirochaetes[col] = data_Spirochaetes[col].astype('float32')



data_Spirochaetes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1940 entries, 0 to 1939
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9143), float32(1), int16(1), int64(2), int8(2482), object(3)
memory usage: 39.4 MB


## Tenericutes

In [33]:
data_Tenericutes = pd.read_csv('/content/Train Dataset/Ten_train.csv')
data_Tenericutes.info(verbose=False,memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163 entries, 0 to 2162
Columns: 11632 entries, Entry to GO:0001884
dtypes: float64(9215), int64(2414), object(3)
memory usage: 192.9 MB


In [34]:
# reduce any int64 columns
int_cols = data_Tenericutes.select_dtypes(include=['int64']).columns
for col in int_cols:
  if (data_Tenericutes[col].max() < max_val_8bit) and (data_Tenericutes[col].min() > min_val_8bit):
    data_Tenericutes[col] = data_Tenericutes[col].astype('int8')
  elif (data_Tenericutes[col].max() < max_val_16bit) and (data_Tenericutes[col].min() > min_val_16bit):
      data_Tenericutes[col] = data_Tenericutes[col].astype('int16')


# reduce any float64 columns
float_cols = data_Tenericutes.select_dtypes(include=['float64']).columns
for col in float_cols:
  if (data_Tenericutes[col].max() < max_val_16bit) and (data_Tenericutes[col].min() > min_val_16bit):
      data_Tenericutes[col] = data_Tenericutes[col].astype('float16')
  elif (data_Tenericutes[col].max() < max_val_32bit) and (data_Tenericutes[col].min() > min_val_32bit):
    data_Tenericutes[col] = data_Tenericutes[col].astype('float32')



data_Tenericutes.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163 entries, 0 to 2162
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(9214), float32(1), int16(1), int64(2), int8(2411), object(3)
memory usage: 44.1 MB


In [35]:
del min_val_8bit, max_val_8bit, min_val_16bit, max_val_16bit, min_val_32bit, max_val_32bit

## Combine all into one dataset

In [36]:
df = pd.concat([data_Actinobacteria1, data_Actinobacteria2,
                data_Bacteroidetes, data_Chlamydiae,
                data_Cyanobacteria, data_Firmicutes1,
                data_Firmicutes2, data_Fusobacteria,
                data_Proteobacteria, data_Proteobacteria4,
                data_Proteobacteria5, data_Spirochaetes,
                data_Tenericutes
                ])
df.info(verbose=False,memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 47746 entries, 0 to 2162
Columns: 11632 entries, Entry to GO:0001884
dtypes: float16(11625), float32(2), float64(2), object(3)
memory usage: 1.1 GB


In [37]:
del data_Actinobacteria1, data_Actinobacteria2, data_Bacteroidetes, data_Chlamydiae, data_Cyanobacteria, data_Firmicutes1, data_Firmicutes2, data_Fusobacteria, data_Proteobacteria, data_Proteobacteria4, data_Proteobacteria5, data_Spirochaetes, data_Tenericutes

## Remove duplicates

In [38]:
df.drop_duplicates(inplace=True)

## Separate into X and y

In [39]:
# In the paper there are 9890 features so drop the entry, entry name and sequence column
#  the binary target labels are the GO columns

go_columns = [col for col in df.columns if col.startswith('GO')]
y = df[go_columns]
X = df.drop(go_columns, axis=1)
X.drop(['Entry','Entry name','Sequence'], axis=1,inplace=True)


In [40]:
# df not needed
del df

# There will be 3 feature sets
 * f1 contains the entire set of 9,890
 * f2 contains the set of 8,420 features that contained only compositions of amino acids, dipeptides, and tripeptides.
 * f3 contains the remaining 1,470 features that are part of f1 but not part of f2 consisting of various properties and characteristics derived from proteins



## Working with the F1

In [41]:
X.head(10)

,A,C,D,E,F,G,H,I,K,L,...,[LIV]-[LIVMGSTC]-[DET]-[RH]-[FYHCS]-x(2)-S-[GSTNP]-x-[AVC]-[FY]-[STANQ],E-[ST]-[EA]-R-E-A-[RK]-x-[LI],[EQ]-[YF]-A-[LIVM]-x(2)-[LIVM]-x(4)-[LIVMF](3)-x-G-H-x(2)-C-G,[RK]-x-[STA]-x(2)-S-x-C-Y-[SL],[GSTAP]-x(2)-[DNEQA]-[LIVM]-[GSA]-x(2)-[LIVMFYT]-[GAN]-[LIVMST]-[ST]-x(6)-R-[LIVT]-x(2)-[LIVM]-x(3)-G,[LIVMFY](2)-D-[STA]-H-x-H-[LIVMFP]-[DN],"[GA]-x(1,2)-[DE]-x-Y-x-[STAPV]-x-C-[NKR]-x-[CH]-[LIVMFYWH]",K-x-[AV]-x(4)-G-x(2)-[LIVT]-x-V-P-x(2)-[LIVC]-x(2)-[GD],"[MFY]-x-G-H-G-[LIVMC]-[GSHN]-x(3)-H-x(4)-[LIVM]-x(1,2)-[HN]-[YWVHF]",P-x(2)-R-G-[STAIV](2)-x-N-[APK]-x-[DE]
0,0.107117,0.005100,0.107117,0.096924,0.035706,0.086731,0.040802,0.035706,0.025513,0.107117,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.125732,0.005463,0.081970,0.098389,0.027328,0.071045,0.038239,0.032776,0.021851,0.120239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.146484,0.006371,0.082825,0.095520,0.012741,0.082825,0.050964,0.025482,0.006371,0.127441,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.089905,0.005619,0.089905,0.112366,0.028091,0.056183,0.033722,0.022476,0.028091,0.123596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.095520,0.005619,0.095520,0.112366,0.028091,0.073059,0.033722,0.028091,0.016861,0.117981,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.142578,0.003565,0.053467,0.058838,0.008911,0.087341,0.021393,0.016037,0.012474,0.133667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.088867,0.005554,0.094421,0.105530,0.027771,0.072205,0.038879,0.033325,0.022217,0.122192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.166626,0.009262,0.037048,0.101868,0.018524,0.064819,0.009262,0.027771,0.027771,0.074097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.103027,0.015266,0.045807,0.095398,0.019089,0.080139,0.015266,0.045807,0.015266,0.110657,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.116943,0.022385,0.039795,0.057220,0.027359,0.052246,0.012436,0.029846,0.064697,0.084595,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert to Numpy Array

In [42]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [43]:
scaler = StandardScaler()
f1 = scaler.fit_transform(X)

In [44]:
y.shape

(47746, 1739)

In [45]:
# 1% neurons in the hidden layer of the input + output layer
neuron_count = int(round(((9890)+1739)*0.01,0))
neuron_count

116

In [47]:
# implement a neural network with a single hidden layer with 1 neuron
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import HeUniform

# our input is a (9864, 9890) array 9890 features and 9864 records
# we want to flatten it

# hidden layer has 1% neurons of the input+output and activation taken as relu

# output layer has 1739 neurons as there are 1739 binary labels

m1 = Sequential(
     [
    Dense(units=neuron_count, activation="relu", kernel_initializer=HeUniform(), input_dim=9890),
    Dense(units=1739, activation="sigmoid")  # output layer with 1739 neurons
])




## Metrics Used in the study
1. Precision
2. Recall
3. F1 Score
4. Micro Average
5. Macro Average
6. Weighted Average
7. Samples Average
8. Zero One Loss
9. Hamming Loss
10. Matthew's Correlaton Coefficent
11. Consolidated Performance Metric

In [48]:
from sklearn.metrics import hamming_loss, zero_one_loss, matthews_corrcoef, precision_recall_fscore_support

def compute_metrics(y_true, y_pred):
    y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary values

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_binary, average=None)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='micro')
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='weighted')
    precision_samples, recall_samples, f1_samples, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='samples')

    hamming = hamming_loss(y_true, y_pred_binary)
    zero_one = zero_one_loss(y_true, y_pred_binary)
    mcc = matthews_corrcoef(y_true.ravel(), y_pred_binary.ravel())

    return {
        "precision": precision, "recall": recall, "f1": f1,
        "precision_micro": precision_micro, "recall_micro": recall_micro, "f1_micro": f1_micro,
        "precision_macro": precision_macro, "recall_macro": recall_macro, "f1_macro": f1_macro,
        "precision_weighted": precision_weighted, "recall_weighted": recall_weighted, "f1_weighted": f1_weighted,
        "precision_samples": precision_samples, "recall_samples": recall_samples, "f1_samples": f1_samples,
        "hamming_loss": hamming, "zero_one_loss": zero_one, "mcc": mcc
    }


In [50]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

m1.compile(
    optimizer = Adam(learning_rate=0.00001),
    loss = BinaryCrossentropy(),
    metrics = ['Precision', 'Recall']
    )


In [51]:
m1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 116)                 │       1,147,356 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1739)                │         203,463 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,350,819 (5.15 MB)

 Trainable params: 1,350,819 (5.15 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
m1.fit(f1,
       y,
       epochs=100,
       batch_size=50
       )

In [ ]:
m1.save("model.keras")

In [ ]:
import os



print(f"Model file size: {os.path.getsize('/content/model.keras') / (1024 * 1024):.2f} MB")


Model file size: 15.48 MB


In [ ]:
m1

<Sequential name=sequential, built=True>